In [ ]:
%pip install pyfqmr
%pip install trimesh
%pip install pymeshlab
%pip install open3d plotly # mesh visualization

In [ ]:
import numpy as np
import os
import open3d as o3d
import plotly.graph_objects as go
import glob

def mesh_visualization(model_path):

  mesh_path = glob.glob(os.path.join(model_path))[0]
  mesh = o3d.io.read_triangle_mesh(mesh_path)
  if not mesh.has_vertex_normals():
    mesh.compute_vertex_normals()
  if not mesh.has_triangle_normals():
    mesh.compute_triangle_normals()

  triangles = np.asarray(mesh.triangles)
  vertices = np.asarray(mesh.vertices)
  colors = None
  if mesh.has_triangle_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
    colors = tuple(map(tuple, colors))
  else:
    colors = (1.0, 0.0, 0.0)
  fig = go.Figure(
    data=[
      go.Mesh3d(
        x=vertices[:,0],
        y=vertices[:,1],
        z=vertices[:,2],
        i=triangles[:,0],
        j=triangles[:,1],
        k=triangles[:,2],
        facecolor=colors,
        opacity=0.50)
    ],
    layout=dict(
      scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
      )
    )
  )
  fig.show()


In [ ]:
import pymeshlab as ml

model_path = "model.obj"

ms = ml.MeshSet()
ms.load_new_mesh(model_path)
m = ms.current_mesh()
print('input mesh has', m.vertex_number(), 'vertex and', m.face_number(), 'faces')

mesh_visualization(model_path)

In [ ]:
#Target number of vertex
TARGET=1000

#Estimate number of faces to have 100+10000 vertex using Euler
numFaces = 100 + 2*TARGET

#Simplify the mesh. Only first simplification will be agressive
while (ms.current_mesh().vertex_number() > TARGET):
    ms.apply_filter('simplification_quadric_edge_collapse_decimation', targetfacenum=numFaces, preservenormal=True)
    print("Decimated to", numFaces, "faces mesh has", ms.current_mesh().vertex_number(), "vertex")
    #Refine our estimation to slowly converge to TARGET vertex number
    numFaces = numFaces - (ms.current_mesh().vertex_number() - TARGET)

m = ms.current_mesh()
print('output mesh has', m.vertex_number(), 'vertex and', m.face_number(), 'faces')
ms.save_current_mesh('output.obj')

mesh_visualization(model_path = 'output.obj')

In [ ]:
import pyfqmr
import trimesh as tr

#Target number of vertex
TARGET=1000
AGGRESSIVENESS =7
VERBOSE = 10

example = tr.load_mesh('model.obj')
example

In [ ]:
mesh_simplifier = pyfqmr.Simplify()
mesh_simplifier.setMesh(example.vertices, example.faces)

In [ ]:
vertices, faces, normals = mesh_simplifier.getMesh()
print(len(vertices), len(faces), len(normals))

In [ ]:
mesh_simplifier.simplify_mesh(target_count = TARGET, aggressiveness=AGGRESSIVENESS, preserve_border=True, verbose=VERBOSE)

In [ ]:
vertices, faces, normals = mesh_simplifier.getMesh()
print(len(vertices), len(faces), len(normals))